# 계층적 맥락 보존 RAG 시스템

이 노트북은 문서의 계층적 구조를 보존하여 검색 시 상위 맥락 정보를 포함한 정확한 검색 결과를 제공하는 시스템을 구현합니다.

## 해결하는 문제
- 단순 제목만으로는 맥락 파악이 어려운 문제
- 상위 섹션 정보가 손실되어 잘못된 정보 제공 위험
- purchaseState 같은 범용 용어의 모듈별 차이점 구분

In [8]:
import re
import uuid
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass, field
from pathlib import Path
import json

from langchain.docstore.document import Document
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.storage import InMemoryStore

## 1. 데이터 구조 정의

In [9]:
@dataclass
class HierarchicalSection:
    """계층적 섹션 정보를 담는 클래스"""
    id: str
    level: int  # 헤더 레벨 (1, 2, 3, ...)
    title: str
    full_path: str  # 전체 경로 (예: "SDK > API Specification")
    content: str
    parent_id: Optional[str] = None
    children: List[str] = field(default_factory=list)  # 하위 섹션 ID들
    metadata: Dict[str, Any] = field(default_factory=dict)
    start_line: int = 0  # 섹션 시작 라인 번호


@dataclass
class ContextualDocument:
    """맥락 정보가 포함된 문서"""
    id: str
    content: str
    section_path: str  # 전체 섹션 경로
    section_hierarchy: List[str]  # 계층 구조 (["SDK", "API Specification"])
    parent_context: str  # 상위 맥락 정보
    metadata: Dict[str, Any] = field(default_factory=dict)

## 2. 계층적 맥락 RAG 시스템 구현

In [10]:
class HierarchicalContextRAG:
    """계층적 맥락을 보존하는 RAG 시스템"""
    
    def __init__(self, embedding_model: str = "bge-m3:latest"):
        self.embedding_model = embedding_model
        self.sections: Dict[str, HierarchicalSection] = {}
        self.documents: List[ContextualDocument] = []
        self.vector_store: Optional[FAISS] = None
        self.bm25_retriever: Optional[BM25Retriever] = None
        self.ensemble_retriever: Optional[EnsembleRetriever] = None
        
    def parse_markdown_hierarchy(self, md_text: str, doc_id: str) -> List[HierarchicalSection]:
        """마크다운을 계층적 구조로 파싱"""
        lines = md_text.splitlines()
        sections = []
        current_hierarchy = []  # 현재까지의 계층 구조
        current_section = None
        
        for i, line in enumerate(lines):
            # 헤더 매칭
            header_match = re.match(r'^(#{1,6})\s+(.+)$', line)
            if header_match:
                level = len(header_match.group(1))
                title = header_match.group(2).strip()
                
                # 계층 구조 업데이트
                if level == 1:
                    current_hierarchy = [title]
                elif level <= len(current_hierarchy):
                    current_hierarchy = current_hierarchy[:level-1] + [title]
                else:
                    current_hierarchy.append(title)
                
                # 이전 섹션 완료
                if current_section:
                    current_section.content = '\n'.join(lines[current_section.start_line:i])
                    sections.append(current_section)
                
                # 새 섹션 생성
                section_id = str(uuid.uuid4())
                current_section = HierarchicalSection(
                    id=section_id,
                    level=level,
                    title=title,
                    full_path=' > '.join(current_hierarchy),
                    content='',
                    start_line=i+1
                )
                
                # 부모-자식 관계 설정
                if level > 1 and len(current_hierarchy) > 1:
                    parent_path = ' > '.join(current_hierarchy[:-1])
                    for section in sections:
                        if section.full_path == parent_path:
                            current_section.parent_id = section.id
                            section.children.append(section_id)
                            break
        
        # 마지막 섹션 처리
        if current_section:
            current_section.content = '\n'.join(lines[current_section.start_line:])
            sections.append(current_section)
        
        return sections
    
    def create_contextual_documents(self, sections: List[HierarchicalSection]) -> List[ContextualDocument]:
        """계층적 섹션을 맥락 정보가 포함된 문서로 변환"""
        contextual_docs = []
        
        for section in sections:
            # 상위 맥락 정보 생성
            parent_context = self._build_parent_context(section, sections)
            
            # 계층 구조 생성
            hierarchy = section.full_path.split(' > ')
            
            # 문서 생성
            doc = ContextualDocument(
                id=section.id,
                content=section.content,
                section_path=section.full_path,
                section_hierarchy=hierarchy,
                parent_context=parent_context,
                metadata={
                    'section_title': section.title,
                    'section_level': section.level,
                    'parent_id': section.parent_id,
                    'children': section.children
                }
            )
            contextual_docs.append(doc)
        
        return contextual_docs
    
    def _build_parent_context(self, section: HierarchicalSection, all_sections: List[HierarchicalSection]) -> str:
        """상위 맥락 정보 구축"""
        if not section.parent_id:
            return section.title
        
        parent_contexts = []
        current_section = section
        
        # 상위 섹션들을 거슬러 올라가며 맥락 구축
        while current_section.parent_id:
            for parent in all_sections:
                if parent.id == current_section.parent_id:
                    parent_contexts.insert(0, f"{parent.title}: {parent.content[:200]}...")
                    current_section = parent
                    break
            else:
                break
        
        return " | ".join(parent_contexts)
    
    def build_search_documents(self, contextual_docs: List[ContextualDocument]) -> List[Document]:
        """검색용 Document 객체 생성"""
        search_docs = []
        
        for doc in contextual_docs:
            # 검색 텍스트에 맥락 정보 포함
            search_content = f"""
제목: {doc.section_path}
상위 맥락: {doc.parent_context}

내용:
{doc.content}
""".strip()
            
            # 메타데이터에 계층 정보 포함
            metadata = {
                **doc.metadata,
                'section_path': doc.section_path,
                'section_hierarchy': doc.section_hierarchy,
                'parent_context': doc.parent_context,
                'context_level': len(doc.section_hierarchy)
            }
            
            search_docs.append(Document(
                page_content=search_content,
                metadata=metadata
            ))
        
        return search_docs
    
    def build_retrievers(self, search_docs: List[Document]):
        """검색기 구축"""
        # 벡터 검색
        embeddings = OllamaEmbeddings(model=self.embedding_model)
        self.vector_store = FAISS.from_documents(
            search_docs, 
            embeddings, 
            distance_strategy=DistanceStrategy.COSINE
        )
        vector_retriever = self.vector_store.as_retriever(search_kwargs={"k": 10})
        
        # BM25 검색 (맥락 정보 강화)
        bm25_docs = []
        for doc in search_docs:
            # BM25용으로 맥락 정보를 더 강화
            enhanced_content = f"""
{doc.page_content}

추가 맥락:
- 섹션 경로: {doc.metadata.get('section_path', '')}
- 계층 구조: {' > '.join(doc.metadata.get('section_hierarchy', []))}
- 상위 맥락: {doc.metadata.get('parent_context', '')}
""".strip()
            
            bm25_docs.append(Document(
                page_content=enhanced_content,
                metadata=doc.metadata
            ))
        
        self.bm25_retriever = BM25Retriever.from_documents(bm25_docs)
        self.bm25_retriever.k = 10
        
        # 앙상블 검색기
        self.ensemble_retriever = EnsembleRetriever(
            retrievers=[self.bm25_retriever, vector_retriever],
            weights=[0.6, 0.4]  # BM25에 더 높은 가중치 (맥락 정보 활용)
        )
    
    def search_with_context(self, query: str, k: int = 5) -> List[Dict[str, Any]]:
        """맥락을 고려한 검색"""
        if not self.ensemble_retriever:
            raise ValueError("검색기가 구축되지 않았습니다. build_retrievers()를 먼저 호출하세요.")
        
        # 검색 실행
        docs = self.ensemble_retriever.get_relevant_documents(query)
        
        # 결과에 맥락 정보 추가
        results = []
        for doc in docs:
            result = {
                'content': doc.page_content,
                'section_path': doc.metadata.get('section_path', ''),
                'section_hierarchy': doc.metadata.get('section_hierarchy', []),
                'parent_context': doc.metadata.get('parent_context', ''),
                'context_level': doc.metadata.get('context_level', 0),
                'metadata': doc.metadata
            }
            results.append(result)
        
        return results[:k]
    
    def search_by_module(self, query: str, target_module: str, k: int = 5) -> List[Dict[str, Any]]:
        """특정 모듈 내에서만 검색"""
        all_results = self.search_with_context(query, k * 3)  # 더 많은 결과 가져오기
        
        # 대상 모듈로 필터링
        filtered_results = []
        for result in all_results:
            hierarchy = result['section_hierarchy']
            if hierarchy and hierarchy[0].lower() == target_module.lower():
                filtered_results.append(result)
                if len(filtered_results) >= k:
                    break
        
        return filtered_results
    
    def compare_across_modules(self, query: str, modules: List[str]) -> Dict[str, List[Dict[str, Any]]]:
        """여러 모듈에서 동일한 쿼리로 검색하여 비교"""
        comparison = {}
        
        for module in modules:
            results = self.search_by_module(query, module, k=3)
            comparison[module] = results
        
        return comparison

## 3. 샘플 문서 생성

In [11]:
def create_sample_document() -> str:
    """테스트용 샘플 문서 생성"""
    return """
# SDK
## 개요
SDK는 Software Development Kit의 약자로, 개발자가 쉽게 애플리케이션을 개발할 수 있도록 도와주는 도구 모음입니다.

## API Specification
### purchaseState
purchaseState는 결제 상태를 나타내는 열거형입니다.

**Enum 값:**
- SUCCEED: 결제 성공
- FAILED: 결제 실패

### 사용 예시
```java
if (purchaseState == PurchaseState.SUCCEED) {
    // 결제 성공 처리
}
```

## 주의사항
- purchaseState는 결제 완료 후에만 정확한 값을 반환합니다.
- 네트워크 오류 시 FAILED가 반환될 수 있습니다.

# Server to Server API
## 개요
Server to Server API는 서버 간 통신을 위한 RESTful API입니다.

## API Specification
### purchaseState
purchaseState는 결제 상태를 나타내는 JSON 필드입니다.

**JSON 값:**
- COMPLETED: 결제 완료
- FAILED: 결제 실패  
- PROCESSING: 결제 처리 중

### API 응답 예시
```json
{
  "purchaseState": "COMPLETED",
  "transactionId": "12345",
  "amount": 1000
}
```

## 주의사항
- PROCESSING 상태는 일시적이며, 최종적으로 COMPLETED 또는 FAILED로 변경됩니다.
- 웹훅을 통해 상태 변경을 실시간으로 받을 수 있습니다.
"""

# 샘플 문서 생성
sample_doc = create_sample_document()
print("📄 샘플 문서 생성 완료")
print(f"문서 길이: {len(sample_doc)} 문자")

📄 샘플 문서 생성 완료
문서 길이: 819 문자


## 4. 시스템 초기화 및 문서 파싱

In [12]:
# RAG 시스템 초기화
rag = HierarchicalContextRAG()
print("🚀 계층적 맥락 RAG 시스템 초기화 완료")

# 마크다운 계층적 파싱
sections = rag.parse_markdown_hierarchy(sample_doc, "sample_doc")
print(f"📋 계층적 섹션 파싱 완료: {len(sections)}개 섹션")

# 파싱된 섹션 정보 출력
print("\n📂 파싱된 섹션 구조:")
for section in sections:
    print(f"  {'  ' * (section.level-1)}• {section.title} (레벨 {section.level})")
    print(f"    경로: {section.full_path}")
    if section.content.strip():
        print(f"    내용 미리보기: {section.content[:100]}...")
    print("-" * 100)

🚀 계층적 맥락 RAG 시스템 초기화 완료
📋 계층적 섹션 파싱 완료: 12개 섹션

📂 파싱된 섹션 구조:
  • SDK (레벨 1)
    경로: SDK
----------------------------------------------------------------------------------------------------
    • 개요 (레벨 2)
    경로: SDK > 개요
    내용 미리보기: SDK는 Software Development Kit의 약자로, 개발자가 쉽게 애플리케이션을 개발할 수 있도록 도와주는 도구 모음입니다.
...
----------------------------------------------------------------------------------------------------
    • API Specification (레벨 2)
    경로: SDK > API Specification
----------------------------------------------------------------------------------------------------
      • purchaseState (레벨 3)
    경로: SDK > API Specification > purchaseState
    내용 미리보기: purchaseState는 결제 상태를 나타내는 열거형입니다.

**Enum 값:**
- SUCCEED: 결제 성공
- FAILED: 결제 실패
...
----------------------------------------------------------------------------------------------------
      • 사용 예시 (레벨 3)
    경로: SDK > API Specification > 사용 예시
    내용 미리보기: ```java
if (purchaseState == PurchaseState.SUCCEED) {
    // 결제 성공 처리

In [13]:
def read_file_content(input_path: str) -> str:
    try:
        # UTF-8 인코딩으로 파일 읽기
        with open(input_path, 'r', encoding='utf-8') as f:
            content = f.read()
        return content
    except FileNotFoundError:
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {input_path}")
      
src = read_file_content("data/dev_center_guide_allmd_touched.md")

# RAG 시스템 초기화
rag = HierarchicalContextRAG()
print("🚀 계층적 맥락 RAG 시스템 초기화 완료")

# 마크다운 계층적 파싱
sections = rag.parse_markdown_hierarchy(src, "dev_center_guide_allmd_touched")
print(f"📋 계층적 섹션 파싱 완료: {len(sections)}개 섹션")

# 파싱된 섹션 정보 출력
print("\n📂 파싱된 섹션 구조:")
for section in sections:
    print(f"  {'  ' * (section.level-1)}• {section.title} (레벨 {section.level})")
    print(f"    경로: {section.full_path}")
    if section.content.strip():
        print(f"    내용 미리보기: {section.content}...")
    print("-" * 100)

🚀 계층적 맥락 RAG 시스템 초기화 완료
📋 계층적 섹션 파싱 완료: 542개 섹션

📂 파싱된 섹션 구조:
  • 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드 (레벨 1)
    경로: 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드
    내용 미리보기: 
원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.

보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.

{% hint style="info" %}
API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 [여기](old-version/v16)를 클릭해주세요.
{% endhint %}

{% hint style="info" %}
현재 판매중인 앱을 대한민국 외 국가/지역으로 배포하기 위해서는 아래 가이드를 참고해주세요

* [대한민국 외 국가 및 지역 배포를 위한 가이드](../glb)
{% endhint %}

If you are comfortable with English, please change the language to English from the upper left side in this page.

* [01. 원스토어 인앱결제 개요](v21/ov)
* [02. 원스토어 인앱결제 적용을 위한 사전준비](v21/pre)
* [03. 결제 테스트 및 보안](v21/test)
* [04. 원스토어 인앱결제 SDK를 사용해 구현하기](v21/sdk)
* [05. 원스토어 인앱결제 레퍼런스](v21/references)
* [06. 원스토어 인앱결제 서버 API (API V7)](v21/serverapi)
* [07. PNS(Payment Notification Service) 이용하기](v21/pns)
* [08. 정기 결제 적용하기](v21/subs)
* [09. 원스토어 인앱결제 릴리즈 노트](v21/releasenote)
* [10. Sam

## 5. 맥락 정보가 포함된 문서 생성

In [14]:
# 맥락 정보가 포함된 문서 생성
contextual_docs = rag.create_contextual_documents(sections)
print(f"📝 맥락 문서 생성 완료: {len(contextual_docs)}개 문서")

# 맥락 문서 정보 출력
print("\n📄 맥락 문서 정보:")
for i, doc in enumerate(contextual_docs, 1):
    print(f"\n--- 문서 {i} ---")
    print(f"섹션 경로: {doc.section_path}")
    print(f"계층 구조: {doc.section_hierarchy}")
    print(f"상위 맥락: {doc.parent_context[:100]}...")
    print(f"내용 길이: {len(doc.content)} 문자")

📝 맥락 문서 생성 완료: 542개 문서

📄 맥락 문서 정보:

--- 문서 1 ---
섹션 경로: 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드
계층 구조: ['원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드']
상위 맥락: 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드...
내용 길이: 1159 문자

--- 문서 2 ---
섹션 경로: 01. 원스토어 인앱결제 개요
계층 구조: ['01. 원스토어 인앱결제 개요']
상위 맥락: 01. 원스토어 인앱결제 개요...
내용 길이: 0 문자

--- 문서 3 ---
섹션 경로: 01. 원스토어 인앱결제 개요 > **원스토어 인앱결제란?** <a href="#id-01." id="id-01."></a>
계층 구조: ['01. 원스토어 인앱결제 개요', '**원스토어 인앱결제란?** <a href="#id-01." id="id-01."></a>']
상위 맥락: 01. 원스토어 인앱결제 개요: ......
내용 길이: 307 문자

--- 문서 4 ---
섹션 경로: 01. 원스토어 인앱결제 개요 > **인앱 상품의 유형** <a href="#id-01." id="id-01."></a>
계층 구조: ['01. 원스토어 인앱결제 개요', '**인앱 상품의 유형** <a href="#id-01." id="id-01."></a>']
상위 맥락: 01. 원스토어 인앱결제 개요: ......
내용 길이: 895 문자

--- 문서 5 ---
섹션 경로: 01. 원스토어 인앱결제 개요 > **결제 프로세스** <a href="#id-01." id="id-01."></a>
계층 구조: ['01. 원스토어 인앱결제 개요', '**결제 프로세스** <a href="#id-01." id="id-01."></a>']
상위 맥락: 01. 원스토어 인앱결제 개요: ......
내용 길이: 47 문자

--- 문서 6 ---
섹션 경로: 01. 원스토어 인앱결제 개요 > *

## 6. 검색 문서 생성 및 검색기 구축

In [16]:
# 검색용 문서 생성
search_docs = rag.build_search_documents(contextual_docs)
print(f"🔍 검색 문서 생성 완료: {len(search_docs)}개 문서")

# 검색 문서 예시 출력
print("\n📋 검색 문서 예시:")
for i, doc in enumerate(search_docs[:2], 1):
    print(f"\n--- 검색 문서 {i} ---")
    print(f"메타데이터: {doc.metadata}")
    print(f"내용 미리보기:\n{doc.page_content[:300]}...")

# 검색기 구축
rag.build_retrievers(search_docs)
print("\n🔧 검색기 구축 완료")

🔍 검색 문서 생성 완료: 542개 문서

📋 검색 문서 예시:

--- 검색 문서 1 ---
메타데이터: {'section_title': '원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드', 'section_level': 1, 'parent_id': None, 'children': [], 'section_path': '원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드', 'section_hierarchy': ['원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드'], 'parent_context': '원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드', 'context_level': 1}
내용 미리보기:
제목: 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드
상위 맥락: 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드

내용:

원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.

보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.

{% hint style="info" %}
API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 [여기](old-version/v16)를 클릭해주세요.
{...

--- 검색 문서 2 ---
메타데이터: {'section_title': '01. 원스토어 인앱결제 개요', 'section_level': 1, 'parent_id': None, 'children': ['f9d9850f-7249-4a2e-ae3c-474ce2108d46', '3de25d28-22b5-405c-bba8-634919979995', '21e067d8-4834-44e9-a38a-1a6a186ecec0', '76114c85-de89-4135-94ab-eb00b9733785'], 'section_path': '01. 원스토어 인앱결제 개요'

## 7. 검색 테스트

In [17]:
print("="*60)
print("🧪 검색 테스트")
print("="*60)

# 테스트 1: 일반 검색
print("\n1️⃣ 일반 검색: 'purchaseState는 어떤 값들이 있나요?'")
results = rag.search_with_context("purchaseState는 어떤 값들이 있나요?", k=3)

for i, result in enumerate(results, 1):
    print(f"\n--- 결과 {i} ---")
    print(f"📂 섹션: {result['section_path']}")
    print(f"📄 내용: {result['content']}...")

🧪 검색 테스트

1️⃣ 일반 검색: 'purchaseState는 어떤 값들이 있나요?'

--- 결과 1 ---
📂 섹션: 01. 원스토어 인앱결제 개요 > **인앱 상품의 유형** <a href="#id-01." id="id-01."></a>
📄 내용: 제목: 01. 원스토어 인앱결제 개요 > **인앱 상품의 유형** <a href="#id-01." id="id-01."></a>
상위 맥락: 01. 원스토어 인앱결제 개요: ...

내용:

원스토어 인앱결제 API V7(SDK V21)는 관리형 상품과 구독형 상품. 두 가지 유형의 인앱 상품을 제공 합니다.\
우선, 각 타입이 어떤 특성을 가지고 있는지 확인하신 후 본인이 제공하는 상품에 맞게 인앱 상품 목록을 구성해 보시기 바랍니다.

<table><thead><tr><th width="239">구분</th><th>내용</th></tr></thead><tbody><tr><td>관리형 상품</td><td>구매 후 소비(consume)하기 전까지 재구매가 되지 않는 상품입니다. 해당 상품을 다시 구매하기 위해서는 소비(consume) 처리를 해야 합니다.<br>해당 특성을 이용하여 소비성/영구성/기간제 형태의 인앱 상품을 제공 할 수 있습니다.</td></tr><tr><td>월정액 상품 (Deprecated)</td><td><p>한 번 가입하면 지정된 날짜에 매월 일정 금액이 자동결제 되는 상품으로, 자동결제 갱신(재결제)은 원스토어에서 처리합니다.</p><p>SDK V21 (API V7) 적용 이후 신규 월정액 상품은 만들 수 없습니다. (기존에 만들어진 월정액 상품은 계속 이용 하실 수 있습니다.) </p><p>구독형 상품을 이용하여 정기 결제 상품을 제공할 수 있습니다,  </p></td></tr><tr><td>구독형 상품</td><td><p>SDK V21 (API V7) 부터 제공되는 정기 결제 상품 타입입니다.</p><p>정해진 주기에 따라 원스토어에서 정기 결제를 처리하며, 신규 고객을 유치하기 

In [18]:
# 테스트 2: SDK 모듈에서만 검색
print("\n2️⃣ SDK 모듈에서만 검색: 'purchaseState 값'")
sdk_results = rag.search_by_module("purchaseState 값", "SDK", k=2)

for i, result in enumerate(sdk_results, 1):
    print(f"\n--- SDK 결과 {i} ---")
    print(f"📂 섹션: {result['section_path']}")
    print(f"📄 내용: {result['content'][:200]}...")


2️⃣ SDK 모듈에서만 검색: 'purchaseState 값'


In [19]:
# 테스트 3: Server to Server API 모듈에서만 검색
print("\n3️⃣ Server to Server API 모듈에서만 검색: 'purchaseState 값'")
server_results = rag.search_by_module("purchaseState 값", "Server to Server API", k=2)

for i, result in enumerate(server_results, 1):
    print(f"\n--- Server to Server API 결과 {i} ---")
    print(f"📂 섹션: {result['section_path']}")
    print(f"📄 내용: {result['content']}...")


3️⃣ Server to Server API 모듈에서만 검색: 'purchaseState 값'


In [20]:
# 테스트 4: 모듈 간 비교
print("\n4️⃣ 모듈 간 비교: 'purchaseState'")
comparison = rag.compare_across_modules("purchaseState", ["SDK", "Server to Server API"])

for module, results in comparison.items():
    print(f"\n--- {module} 모듈 결과 ---")
    for i, result in enumerate(results, 1):
        print(f"  {i}. {result['section_path']}")
        print(f"     {result['content'][:100]}...")

print("\n✅ 모든 테스트 완료!")


4️⃣ 모듈 간 비교: 'purchaseState'

--- SDK 모듈 결과 ---

--- Server to Server API 모듈 결과 ---

✅ 모든 테스트 완료!


## 8. 실제 문서 적용 예시

In [22]:
# 실제 문서 파일이 있다면 적용 예시
def apply_to_real_document(file_path: str):
    """실제 문서에 계층적 맥락 RAG 적용"""
    print(f"📖 실제 문서 적용: {file_path}")
    
    # 문서 로드
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # RAG 시스템 초기화
    rag = HierarchicalContextRAG()
    
    # 문서 파싱 및 검색기 구축
    sections = rag.parse_markdown_hierarchy(content, Path(file_path).stem)
    contextual_docs = rag.create_contextual_documents(sections)
    search_docs = rag.build_search_documents(contextual_docs)
    rag.build_retrievers(search_docs)
    
    print(f"✅ 문서 처리 완료: {len(sections)}개 섹션, {len(search_docs)}개 검색 문서")
    return rag

# 사용 예시 (실제 파일이 있을 때)
rag_system = apply_to_real_document("data/dev_center_guide_allmd_touched.md")
results = rag_system.search_with_context("PNS의 purchaseState는 어떤 값들이 있나요?", k=3)

for i, result in enumerate(results, 1):
    print(f"\n--- 결과 {i} ---")
    print(f"📂 섹션: {result['section_path']}")
    print(f"    내용: {result['content']}...")
    print("-" * 100)

📖 실제 문서 적용: data/dev_center_guide_allmd_touched.md
✅ 문서 처리 완료: 542개 섹션, 542개 검색 문서

--- 결과 1 ---
📂 섹션: 01. 원스토어 인앱결제 개요 > **인앱 상품의 유형** <a href="#id-01." id="id-01."></a>
    내용: 제목: 01. 원스토어 인앱결제 개요 > **인앱 상품의 유형** <a href="#id-01." id="id-01."></a>
상위 맥락: 01. 원스토어 인앱결제 개요: ...

내용:

원스토어 인앱결제 API V7(SDK V21)는 관리형 상품과 구독형 상품. 두 가지 유형의 인앱 상품을 제공 합니다.\
우선, 각 타입이 어떤 특성을 가지고 있는지 확인하신 후 본인이 제공하는 상품에 맞게 인앱 상품 목록을 구성해 보시기 바랍니다.

<table><thead><tr><th width="239">구분</th><th>내용</th></tr></thead><tbody><tr><td>관리형 상품</td><td>구매 후 소비(consume)하기 전까지 재구매가 되지 않는 상품입니다. 해당 상품을 다시 구매하기 위해서는 소비(consume) 처리를 해야 합니다.<br>해당 특성을 이용하여 소비성/영구성/기간제 형태의 인앱 상품을 제공 할 수 있습니다.</td></tr><tr><td>월정액 상품 (Deprecated)</td><td><p>한 번 가입하면 지정된 날짜에 매월 일정 금액이 자동결제 되는 상품으로, 자동결제 갱신(재결제)은 원스토어에서 처리합니다.</p><p>SDK V21 (API V7) 적용 이후 신규 월정액 상품은 만들 수 없습니다. (기존에 만들어진 월정액 상품은 계속 이용 하실 수 있습니다.) </p><p>구독형 상품을 이용하여 정기 결제 상품을 제공할 수 있습니다,  </p></td></tr><tr><td>구독형 상품</td><td><p>SDK V21 (API V7) 부터 제공되는 정기 결제 상품 타입입니다.</p><p>정해진 주기에 

## 9. 성능 비교 및 개선점

In [ ]:
# 기존 방식 vs 계층적 맥락 방식 비교
def compare_search_methods():
    """검색 방식 비교"""
    print("🔍 검색 방식 비교")
    print("="*50)
    
    # 기존 방식 (단순 제목만)
    print("\n📋 기존 방식 (단순 제목):")
    print("  ❌ '개요' 제목만으로는 맥락 파악 불가")
    print("  ❌ purchaseState가 SDK인지 Server API인지 구분 불가")
    print("  ❌ 잘못된 정보 제공 위험")
    
    # 계층적 맥락 방식
    print("\n✅ 계층적 맥락 방식:")
    print("  ✅ 'SDK > API Specification > purchaseState' 전체 경로 제공")
    print("  ✅ 모듈별 차이점 명확히 구분")
    print("  ✅ 상위 맥락 정보 포함으로 정확성 향상")
    print("  ✅ 모듈별 필터링 검색 가능")

compare_search_methods()